In [ ]:
#!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=c1226f07c5fa2593f5004c349029d06d9eac1623e6f572bc1a52cf4ab5030286
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [287]:
import os
from pyspark.sql import SparkSession
from functools import reduce
from pyspark.sql.functions import min, max,min as min_, max as max_, to_date, year, concat_ws,substring, col, concat, lit
from pyspark.ml.feature import Bucketizer
from itertools import combinations
from pyspark.sql.types import StructType, StructField, StringType, IntegerType


# Create a SparkSession
spark = SparkSession.builder.appName("ReadCSV").config("spark.sql.legacy.timeParserPolicy", "CORRECTED").getOrCreate()

current_directory = os.getcwd()
print("Current Directory:", current_directory)

# Define the file name and its path relative to the current directory
file_name = "playstore.csv"

# Create the complete file path using os.path.join()
file_path = os.path.join(current_directory,file_name)
print("File Path:", file_path)
print(os.path.exists(file_path))
df = spark.read.csv(file_path, header=True, inferSchema=True)

Current Directory: C:\Users\Manish\Desktop\Segwise_Data_Engineer
File Path: C:\Users\Manish\Desktop\Segwise_Data_Engineer\playstore.csv
True


In [288]:
columns = ["free", "genre", "minInstalls", "price", "ratings", "adSupported", "containsAds", "reviews", 'releasedDayYear', 'sale', 'score','dateUpdated']
df = df.select(*columns)
df = df.limit(4000)
df.show()

+----+------------+-----------+-----+-------+-----------+-----------+-------+---------------+----+-----+-------------------+
|free|       genre|minInstalls|price|ratings|adSupported|containsAds|reviews|releasedDayYear|sale|score|        dateUpdated|
+----+------------+-----------+-----+-------+-----------+-----------+-------+---------------+----+-----+-------------------+
|   1|Art & Design|    1000000|  0.0|  30650|          1|          1|    152|   Sep 27, 2014|   0| 4.75|2019-04-22 08:18:36|
|   1|Art & Design|    1000000|  0.0|  15150|          1|          1|     62|   Nov 13, 2014|   0| 4.52|2019-04-22 08:27:46|
|   1|Art & Design|     500000|  0.0|   3312|          1|          1|     59|   Dec 14, 2013|   0| 3.61|2021-12-23 07:02:19|
|   1|Art & Design|      50000|  0.0|    915|          1|          1|     58|   Oct 13, 2014|   0|  4.5|2018-09-20 09:29:32|
|   0|Art & Design|       1000| 2.61|     38|          0|          0|      5|    Oct 1, 2013|   0|  4.0|2014-06-19 15:34:02|


In [289]:
# Extracting the year from the 'releasedDayYear' column
df = df.withColumn("releasedYear", substring(col("releasedDayYear"), -4, 4).cast("int"))
df = df.filter((col("releasedYear") >= 1900) & (col("releasedYear") <= 2024))

In [290]:
df = df.withColumn("minInstalls", df["minInstalls"].cast("int"))
df = df.withColumn("releasedYear", df["releasedYear"].cast("int"))
df = df.withColumn("price", df["price"].cast("float"))
df = df.withColumn("ratings", df["ratings"].cast("float"))
df = df.withColumn("score", df["score"].cast("float"))

In [291]:
# Filter rows with null values in any column
df = df.na.drop()
numeric_columns = ["free", "minInstalls", "releasedYear","price", "ratings", "adSupported", "containsAds", "reviews", 'sale', 'score', 'updated']
filter_condition = reduce(
    lambda acc, x: acc & col(x).rlike('^[0-9]*$'),
    numeric_columns[1:],
    col(numeric_columns[0]).rlike('^[0-9]*$')
)

# Filter rows where specified columns do not contain numeric values
df = df.filter(~filter_condition)

In [292]:
df.show(truncate=True)

+----+------------+-----------+-----+-------+-----------+-----------+-------+---------------+----+-----+-------------------+------------+
|free|       genre|minInstalls|price|ratings|adSupported|containsAds|reviews|releasedDayYear|sale|score|        dateUpdated|releasedYear|
+----+------------+-----------+-----+-------+-----------+-----------+-------+---------------+----+-----+-------------------+------------+
|   1|Art & Design|    1000000|  0.0|30650.0|          1|          1|    152|   Sep 27, 2014|   0| 4.75|2019-04-22 08:18:36|        2014|
|   1|Art & Design|    1000000|  0.0|15150.0|          1|          1|     62|   Nov 13, 2014|   0| 4.52|2019-04-22 08:27:46|        2014|
|   1|Art & Design|     500000|  0.0| 3312.0|          1|          1|     59|   Dec 14, 2013|   0| 3.61|2021-12-23 07:02:19|        2013|
|   1|Art & Design|      50000|  0.0|  915.0|          1|          1|     58|   Oct 13, 2014|   0|  4.5|2018-09-20 09:29:32|        2014|
|   0|Art & Design|       1000| 2.

In [293]:
# Get the minimum and maximum values for numerical columns
min_max_values = df.agg(
    min_("ratings").alias("min_ratings"),
    max_("ratings").alias("max_ratings"),
    min_("score").alias("min_score"),
    max_("score").alias("max_score"),
    min_("releasedYear").alias("min_released_year"),
    max_("releasedYear").alias("max_released_year"),
    min_("price").alias("min_price"),
    max_("price").alias("max_price"),
    min_("minInstalls").alias("min_installs"),
    max_("minInstalls").alias("max_installs")
).collect()[0]

In [294]:
# Extract min and max values for each column
min_ratings = min_max_values['min_ratings']
max_ratings = min_max_values['max_ratings']
min_score = min_max_values['min_score']
max_score = min_max_values['max_score']
min_released_year = min_max_values['min_released_year']
max_released_year = min_max_values['max_released_year']
min_price = min_max_values['min_price']
max_price = min_max_values['max_price']
min_installs = min_max_values['min_installs']
max_installs = min_max_values['max_installs']

year_interval = 2
# Create ranges for releasedYear with 2-year intervals
year_ranges = [x for x in range(min_released_year, max_released_year + 3, year_interval)]
print(year_ranges)
# Bucketizing 'releasedYear' column
df = df.filter(~col("releasedYear").isNull())
bucketizer_year = Bucketizer(splits=year_ranges, inputCol="releasedYear", outputCol="year_bucket")
df = bucketizer_year.transform(df)

[2012, 2014, 2016, 2018, 2020, 2022, 2024]


In [295]:
# Create ranges for installs with interval 100000
install_interval = 100000
install_ranges = [x for x in range(min_installs, max_installs + install_interval+1,install_interval)]
print(install_ranges)
# Bucketizing 'minInstalls' column
df = df.filter(~col("minInstalls").isNull())
bucketizer_installs = Bucketizer(splits=install_ranges, inputCol="minInstalls", outputCol="install_bucket")
df = bucketizer_installs.transform(df)

[0, 100000, 200000, 300000, 400000, 500000, 600000, 700000, 800000, 900000, 1000000, 1100000, 1200000, 1300000, 1400000, 1500000, 1600000, 1700000, 1800000, 1900000, 2000000, 2100000, 2200000, 2300000, 2400000, 2500000, 2600000, 2700000, 2800000, 2900000, 3000000, 3100000, 3200000, 3300000, 3400000, 3500000, 3600000, 3700000, 3800000, 3900000, 4000000, 4100000, 4200000, 4300000, 4400000, 4500000, 4600000, 4700000, 4800000, 4900000, 5000000, 5100000, 5200000, 5300000, 5400000, 5500000, 5600000, 5700000, 5800000, 5900000, 6000000, 6100000, 6200000, 6300000, 6400000, 6500000, 6600000, 6700000, 6800000, 6900000, 7000000, 7100000, 7200000, 7300000, 7400000, 7500000, 7600000, 7700000, 7800000, 7900000, 8000000, 8100000, 8200000, 8300000, 8400000, 8500000, 8600000, 8700000, 8800000, 8900000, 9000000, 9100000, 9200000, 9300000, 9400000, 9500000, 9600000, 9700000, 9800000, 9900000, 10000000, 10100000]


In [296]:
# Create ranges for price with interval 5
price_interval = 5
price_ranges = [x  for x in range(int(min_price), int(max_price) + 2,price_interval)]
print(price_ranges)
# Bucketizing 'price' column
df = df.filter(~col("price").isNull())
bucketizer_price = Bucketizer(splits=price_ranges, inputCol="price", outputCol="price_bucket")
df = bucketizer_price.transform(df)

[0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95, 100, 105, 110, 115, 120, 125, 130, 135, 140, 145, 150, 155, 160, 165, 170, 175, 180, 185, 190, 195, 200, 205, 210, 215, 220, 225, 230, 235, 240, 245, 250, 255, 260, 265, 270, 275, 280, 285, 290, 295, 300, 305, 310, 315, 320, 325, 330, 335, 340, 345, 350, 355]


In [297]:
ratings_interval = 10000
ratings_ranges = [x  for x in range(int(min_ratings), int(max_ratings) + ratings_interval+1,ratings_interval)]
print(ratings_ranges)
# Bucketizing 'ratings' column
df = df.filter(~col("ratings").isNull())
bucketizer_ratings = Bucketizer(splits=ratings_ranges, inputCol="ratings", outputCol="ratings_bucket")
df = bucketizer_ratings.transform(df)

[0, 10000, 20000, 30000, 40000, 50000, 60000, 70000, 80000, 90000, 100000, 110000, 120000, 130000, 140000, 150000, 160000, 170000, 180000, 190000, 200000, 210000]


In [298]:
score_interval = 1
score_ranges = [x  for x in range(int(min_score), int(max_score) + score_interval,score_interval)]
print(score_ranges)
# Bucketizing 'score' column
df = df.filter(~col("score").isNull())
bucketizer_score = Bucketizer(splits=score_ranges, inputCol="score", outputCol="score_bucket")
df = bucketizer_score.transform(df)

[0, 1, 2, 3, 4, 5]


In [299]:
df = df.withColumn("year_bucket_range",concat(lit("["), (col("year_bucket")*year_interval+min_released_year).cast("int"), lit("-"),(col("year_bucket")*year_interval+min_released_year).cast("int")+year_interval-1, lit("]")))
df = df.withColumn("price_bucket_range",concat(lit("["), (col("price_bucket")*price_interval+min_price).cast("int"), lit("-"),(col("price_bucket")*price_interval+min_price).cast("int")+price_interval, lit("]")))
df = df.withColumn("install_bucket_range",concat(lit("["), (col("install_bucket")*install_interval+min_installs).cast("int"), lit("-"),(col("install_bucket")*install_interval+min_installs).cast("int")+install_interval, lit("]")))
df = df.withColumn("score_bucket_range",concat(lit("["), (col("score_bucket")*score_interval+min_score).cast("int"), lit("-"),(col("score_bucket")*score_interval+min_score).cast("int")+score_interval, lit("]")))
df = df.withColumn("rating_bucket_range",concat(lit("["), (col("ratings_bucket")*ratings_interval+min_ratings).cast("int"), lit("-"),(col("ratings_bucket")*ratings_interval+min_ratings).cast("int")+ratings_interval, lit("]")))
df.select("score", "score_bucket_range", "price", "price_bucket_range", "releasedYear", "year_bucket_range","minInstalls", "install_bucket_range","ratings","rating_bucket_range").show()

+-----+------------------+-----+------------------+------------+-----------------+-----------+--------------------+-------+-------------------+
|score|score_bucket_range|price|price_bucket_range|releasedYear|year_bucket_range|minInstalls|install_bucket_range|ratings|rating_bucket_range|
+-----+------------------+-----+------------------+------------+-----------------+-----------+--------------------+-------+-------------------+
| 4.75|             [4-5]|  0.0|             [0-5]|        2014|      [2014-2015]|    1000000|   [1000000-1100000]|30650.0|      [30000-40000]|
| 4.52|             [4-5]|  0.0|             [0-5]|        2014|      [2014-2015]|    1000000|   [1000000-1100000]|15150.0|      [10000-20000]|
| 3.61|             [3-4]|  0.0|             [0-5]|        2013|      [2012-2013]|     500000|     [500000-600000]| 3312.0|          [0-10000]|
|  4.5|             [4-5]|  0.0|             [0-5]|        2014|      [2014-2015]|      50000|          [0-100000]|  915.0|          [0-

In [301]:
selected_columns = ["free", "genre","adSupported","containsAds","dateUpdated","year_bucket_range","install_bucket_range", "price_bucket_range", "rating_bucket_range", "score_bucket_range"]
# Define the range of permutation sizes (3 in this case) 
min_comb_size = 1
max_comb_size = 3

# Create an empty DataFrame to hold aggregated results
schema = StructType([
    StructField("property", StringType(), True),
    StructField("count", IntegerType(), True)
])

# Create an empty DataFrame with the defined schema
result_df = spark.createDataFrame(spark.sparkContext.emptyRDD(), schema)

# Loop through different combinations sizes
for comb_size in range(min_comb_size, max_comb_size + 1):
    # Generate combinations of selected_columns for the current perm_size without duplicates
    combinations_without_duplicates = list(combinations(selected_columns, comb_size))

    # Iterate through each combination for the current perm_size
    for combination in combinations_without_duplicates:
        combination_list = list(combination)
        print(combination_list)

        properties = [concat(concat_ws("=", lit(column), col(column)),lit(";")).alias(column) for column in combination_list]
        combined_properties = concat(*properties).alias("property")
        # Aggregate count for each unique combination of properties
        aggregation = df.groupBy(*combination_list).count().withColumnRenamed("count", "count")
        aggregation.show(truncate=False)
        # Union the aggregation results with the result_df
        result_df = result_df.union(aggregation.select(combined_properties, col("count")))

['free']
+----+-----+
|free|count|
+----+-----+
|1   |3855 |
|0   |127  |
+----+-----+

['genre']
+---------------+-----+
|genre          |count|
+---------------+-----+
|Art & Design   |2121 |
|Entertainment  |3    |
|Casual         |1    |
|Personalization|8    |
|Travel & Local |1    |
|Photography    |3    |
|Tools          |1    |
|Events         |2    |
|Auto & Vehicles|1040 |
|Sports         |1    |
|Role Playing   |1    |
|Education      |1    |
|Beauty         |799  |
+---------------+-----+

['adSupported']
+-----------+-----+
|adSupported|count|
+-----------+-----+
|1          |2557 |
|0          |1425 |
+-----------+-----+

['containsAds']
+-----------+-----+
|containsAds|count|
+-----------+-----+
|1          |2557 |
|0          |1425 |
+-----------+-----+

['dateUpdated']
+-------------------+-----+
|dateUpdated        |count|
+-------------------+-----+
|2019-04-22 08:18:36|1    |
|2019-04-22 08:27:46|1    |
|2021-12-23 07:02:19|1    |
|2018-09-20 09:29:32|1    |
|2014-0

+------------+-------------------+-----+
|genre       |dateUpdated        |count|
+------------+-------------------+-----+
|Art & Design|2019-04-22 08:18:36|1    |
|Art & Design|2019-04-22 08:27:46|1    |
|Art & Design|2021-12-23 07:02:19|1    |
|Art & Design|2018-09-20 09:29:32|1    |
|Art & Design|2014-06-19 15:34:02|1    |
|Art & Design|2022-10-01 22:13:44|1    |
|Art & Design|2016-06-07 19:02:44|1    |
|Art & Design|2016-12-16 09:26:20|1    |
|Art & Design|2019-08-22 01:05:03|1    |
|Art & Design|2022-08-16 15:34:22|1    |
|Art & Design|2016-09-12 03:18:39|1    |
|Art & Design|2016-10-04 10:30:37|1    |
|Art & Design|2019-12-04 09:29:24|1    |
|Art & Design|2019-12-04 07:42:36|1    |
|Art & Design|2019-07-26 16:54:35|1    |
|Art & Design|2021-01-07 11:18:20|1    |
|Art & Design|2019-01-10 18:39:21|1    |
|Art & Design|2016-10-18 05:09:48|1    |
|Art & Design|2021-08-25 18:15:56|1    |
|Art & Design|2017-01-29 22:36:14|1    |
+------------+-------------------+-----+
only showing top

+-----------+--------------------+-----+
|adSupported|install_bucket_range|count|
+-----------+--------------------+-----+
|1          |[1000000-1100000]   |14   |
|1          |[500000-600000]     |10   |
|1          |[0-100000]          |2458 |
|0          |[0-100000]          |1411 |
|1          |[100000-200000]     |69   |
|1          |[5000000-5100000]   |5    |
|0          |[100000-200000]     |13   |
|1          |[10000000-10100000] |1    |
|0          |[500000-600000]     |1    |
+-----------+--------------------+-----+

['adSupported', 'price_bucket_range']
+-----------+------------------+-----+
|adSupported|price_bucket_range|count|
+-----------+------------------+-----+
|1          |[0-5]             |2555 |
|0          |[0-5]             |1390 |
|0          |[5-10]            |23   |
|0          |[350-355]         |1    |
|0          |[20-25]           |1    |
|0          |[10-15]           |5    |
|1          |[15-20]           |1    |
|0          |[80-85]           |1    |

['dateUpdated', 'price_bucket_range']
+-------------------+------------------+-----+
|dateUpdated        |price_bucket_range|count|
+-------------------+------------------+-----+
|2019-04-22 08:18:36|[0-5]             |1    |
|2019-04-22 08:27:46|[0-5]             |1    |
|2021-12-23 07:02:19|[0-5]             |1    |
|2018-09-20 09:29:32|[0-5]             |1    |
|2014-06-19 15:34:02|[0-5]             |1    |
|2022-10-01 22:13:44|[5-10]            |1    |
|2016-06-07 19:02:44|[0-5]             |1    |
|2016-12-16 09:26:20|[0-5]             |1    |
|2019-08-22 01:05:03|[0-5]             |1    |
|2022-08-16 15:34:22|[0-5]             |1    |
|2016-09-12 03:18:39|[0-5]             |1    |
|2016-10-04 10:30:37|[0-5]             |1    |
|2019-12-04 09:29:24|[0-5]             |1    |
|2019-12-04 07:42:36|[0-5]             |1    |
|2019-07-26 16:54:35|[0-5]             |1    |
|2021-01-07 11:18:20|[0-5]             |1    |
|2019-01-10 18:39:21|[0-5]             |1    |
|2016-10-18 05:09:48|[

+--------------------+-------------------+-----+
|install_bucket_range|rating_bucket_range|count|
+--------------------+-------------------+-----+
|[1000000-1100000]   |[30000-40000]      |1    |
|[1000000-1100000]   |[10000-20000]      |4    |
|[500000-600000]     |[0-10000]          |9    |
|[0-100000]          |[0-10000]          |3869 |
|[100000-200000]     |[0-10000]          |81   |
|[1000000-1100000]   |[0-10000]          |8    |
|[5000000-5100000]   |[0-10000]          |1    |
|[500000-600000]     |[10000-20000]      |2    |
|[5000000-5100000]   |[20000-30000]      |1    |
|[5000000-5100000]   |[50000-60000]      |1    |
|[5000000-5100000]   |[70000-80000]      |1    |
|[1000000-1100000]   |[20000-30000]      |1    |
|[10000000-10100000] |[200000-210000]    |1    |
|[5000000-5100000]   |[80000-90000]      |1    |
|[100000-200000]     |[10000-20000]      |1    |
+--------------------+-------------------+-----+

['install_bucket_range', 'score_bucket_range']
+--------------------

...

['containsAds', 'year_bucket_range', 'score_bucket_range']
+-----------+-----------------+------------------+-----+
|containsAds|year_bucket_range|score_bucket_range|count|
+-----------+-----------------+------------------+-----+
|1          |[2014-2015]      |[4-5]             |5    |
|1          |[2012-2013]      |[3-4]             |1    |
|0          |[2012-2013]      |[4-5]             |2    |
|0          |[2014-2015]      |[0-1]             |7    |
|1          |[2016-2017]      |[0-1]             |72   |
|1          |[2016-2017]      |[3-4]             |8    |
|1          |[2016-2017]      |[4-5]             |19   |
|0          |[2016-2017]      |[0-1]             |31   |
|1          |[2016-2017]      |[2-3]             |3    |
|1          |[2016-2017]      |[1-2]             |1    |
|0          |[2018-2019]      |[0-1]             |221  |
|0          |[2016-2017]      |[4-5]             |4    |
|1          |[2018-2019]      |[0-1]             |531  |
|1          |[2018-2019]     

['containsAds', 'rating_bucket_range', 'score_bucket_range']
+-----------+-------------------+------------------+-----+
|containsAds|rating_bucket_range|score_bucket_range|count|
+-----------+-------------------+------------------+-----+
|1          |[30000-40000]      |[4-5]             |1    |
|1          |[10000-20000]      |[4-5]             |6    |
|1          |[0-10000]          |[3-4]             |47   |
|1          |[0-10000]          |[4-5]             |126  |
|0          |[0-10000]          |[4-5]             |28   |
|0          |[0-10000]          |[0-1]             |1376 |
|1          |[0-10000]          |[0-1]             |2345 |
|1          |[0-10000]          |[2-3]             |17   |
|1          |[0-10000]          |[1-2]             |9    |
|1          |[20000-30000]      |[4-5]             |2    |
|0          |[0-10000]          |[3-4]             |16   |
|1          |[50000-60000]      |[4-5]             |1    |
|1          |[70000-80000]      |[4-5]             |1 

['dateUpdated', 'install_bucket_range', 'rating_bucket_range']
+-------------------+--------------------+-------------------+-----+
|dateUpdated        |install_bucket_range|rating_bucket_range|count|
+-------------------+--------------------+-------------------+-----+
|2019-04-22 08:18:36|[1000000-1100000]   |[30000-40000]      |1    |
|2019-04-22 08:27:46|[1000000-1100000]   |[10000-20000]      |1    |
|2021-12-23 07:02:19|[500000-600000]     |[0-10000]          |1    |
|2018-09-20 09:29:32|[0-100000]          |[0-10000]          |1    |
|2014-06-19 15:34:02|[0-100000]          |[0-10000]          |1    |
|2022-10-01 22:13:44|[0-100000]          |[0-10000]          |1    |
|2016-06-07 19:02:44|[0-100000]          |[0-10000]          |1    |
|2016-12-16 09:26:20|[0-100000]          |[0-10000]          |1    |
|2019-08-22 01:05:03|[100000-200000]     |[0-10000]          |1    |
|2022-08-16 15:34:22|[1000000-1100000]   |[0-10000]          |1    |
|2016-09-12 03:18:39|[0-100000]         

['year_bucket_range', 'install_bucket_range', 'price_bucket_range']
+-----------------+--------------------+------------------+-----+
|year_bucket_range|install_bucket_range|price_bucket_range|count|
+-----------------+--------------------+------------------+-----+
|[2014-2015]      |[1000000-1100000]   |[0-5]             |2    |
|[2012-2013]      |[500000-600000]     |[0-5]             |1    |
|[2014-2015]      |[0-100000]          |[0-5]             |8    |
|[2012-2013]      |[0-100000]          |[0-5]             |2    |
|[2014-2015]      |[0-100000]          |[5-10]            |1    |
|[2016-2017]      |[0-100000]          |[0-5]             |118  |
|[2016-2017]      |[100000-200000]     |[0-5]             |17   |
|[2016-2017]      |[1000000-1100000]   |[0-5]             |1    |
|[2016-2017]      |[500000-600000]     |[0-5]             |3    |
|[2016-2017]      |[5000000-5100000]   |[0-5]             |1    |
|[2018-2019]      |[0-100000]          |[0-5]             |813  |
|[2018-2

['year_bucket_range', 'rating_bucket_range', 'score_bucket_range']
+-----------------+-------------------+------------------+-----+
|year_bucket_range|rating_bucket_range|score_bucket_range|count|
+-----------------+-------------------+------------------+-----+
|[2014-2015]      |[30000-40000]      |[4-5]             |1    |
|[2014-2015]      |[10000-20000]      |[4-5]             |1    |
|[2012-2013]      |[0-10000]          |[3-4]             |1    |
|[2014-2015]      |[0-10000]          |[4-5]             |2    |
|[2012-2013]      |[0-10000]          |[4-5]             |2    |
|[2014-2015]      |[0-10000]          |[0-1]             |10   |
|[2016-2017]      |[0-10000]          |[0-1]             |103  |
|[2016-2017]      |[0-10000]          |[3-4]             |11   |
|[2016-2017]      |[0-10000]          |[4-5]             |22   |
|[2016-2017]      |[0-10000]          |[2-3]             |3    |
|[2016-2017]      |[0-10000]          |[1-2]             |1    |
|[2018-2019]      |[0-1

In [302]:
# Write the aggregated results of all permutations to a single CSV file
if result_df.count() > 0:
    result_df = result_df.where(col("count")>1)
    try:
        file_name = "output_all_insights_spark.csv"
        file_path = os.path.join(current_directory,file_name)
        result_df.write.csv(file_path, header=True, mode='overwrite')
        print(f"Ouput generated in: {current_directory} with file name: {file_name}.")
    except Exception as e:
        print("Trying to write dataframe in csv after conversion to pandas.")
        final_df = result_df.toPandas()
        file_name = "output_all_insights.csv"
        file_path = os.path.join(current_directory,file_name)
        with open(file_path, mode='w') as file:
            final_df.to_csv(file, index=False)
        print(f"Ouput generated in: {current_directory} with file name: {file_name}.")
        
else:
    print("No data to write. DataFrame is empty.")
spark.stop()

Trying to write dataframe in csv after conversion to pandas.
Ouput generated on C:\Users\Manish\Desktop\Segwise_Data_Engineer with file name output_all_insights.csv.
